# Robots do site
```bash
User-agent: * Allow: /
# verificando conteudo do site


In [13]:
# requisitando se conpondo soup
import requests
from bs4 import BeautifulSoup
import json

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.5845.111 Safari/537.36"
}
url = 'https://www.imobiliariaativa.com.br/'
response = requests.get(url,headers = headers)
html_doc = response.text
soup = BeautifulSoup(html_doc, 'html.parser')
imoveis = soup.find_all("div", class_="muda_card1 ms-2 ms-lg-0 col-12 col-md-7 col-lg-6 col-xl-4 mt-4 d-flex align-self-stretch justify-content-center")
lista_imoveis = []
for imovel in imoveis:
    # Título
    titulo_tag = imovel.find('div', class_='card-titulo')
    h2_tag = titulo_tag.find('h2') if titulo_tag else None
    titulo = h2_tag.text.strip() if h2_tag else ''

    # link
    link_tag = (imovel.find('a'))
    link = link_tag.get('href') if link_tag else ''

    # Valor
    valor_tag = imovel.find('div', class_='card-valores')
    valor_div = valor_tag.find('div') if valor_tag else None
    valor = valor_div.text.strip() if valor_div else ''

    # Local
    local_tag = imovel.find('div', class_='card-bairro-cidade-texto')
    local = local_tag.text.strip() if local_tag else ''

    # Descrição
    descricao_tag = imovel.find('div', class_='card-texto')
    p_tag = descricao_tag.find('p') if descricao_tag else None
    descricao = p_tag.text.strip() if p_tag else ''

    # codigo
    cod =  imovel.find('div', class_='cod-imovel')
    tag_cod = cod.find('strong')
    codigo = tag_cod.text.strip() if tag_cod else ''


    # Informações de quartos, suítes, banheiros, garagem
    qtds_divs = imovel.find_all('div', class_='imo-dad-compl-item')
    detalhes = {}
    for item in qtds_divs:
        texto_tag = item.find('span', class_='imo-dad-compl-item-texto')
        qtd_tag = item.find('span', class_='imo-dad-compl-item-qntd')
        oque = texto_tag.text.strip() if texto_tag else ''
        qtd = qtd_tag.text.strip() if qtd_tag else ''
        detalhes[oque] = qtd

    # Impressão organizada
    print(f'''
####################################################
#  Título: {titulo if len(titulo)<30 else titulo[:30]+'...'}{(' '*(40-len(titulo if len(titulo)<30 else titulo[:30]+'...')))+'#'}
#  Valor:  {valor if len(valor)<30 else valor[:30]+'...'}{(' '*(40-len(valor if len(valor)<30 else valor[:30]+'...')))+'#'}
#  Local:  {local if len(local)<30 else local[:30]+'...'}{(' '*(40-len(local if len(local)<30 else local[:30]+'...')))+'#'}
#  COD:    {codigo if len(codigo)<30 else codigo[:30]+'...'}{(' '*(40-len(codigo if len(codigo)<30 else codigo[:30]+'...')))+'#'}
####################################################
''')
    lista_imoveis.append({
        'titulo':titulo,
        'valor':f'R$ {valor.split('R$ ')[-1]}',
        'local':local,
        'cod':codigo,
        'link':f'{url}/{link}',
        'detalhes':detalhes
    })

json_str = json.dumps(lista_imoveis, ensure_ascii=False, indent=4)
with open('out.json','w') as j:
    j.write(json_str)
print(f'Retornos: {len(lista_imoveis)}')


####################################################
#  Título: Casa - Padrão                           #
#  Valor:  Comprar R$ 300.000,00                   #
#  Local:  Vila Pioneiro - Toledo/PR               #
#  COD:    12532                                   #
####################################################


####################################################
#  Título: Apartamento - Padrão                    #
#  Valor:  Comprar R$ 1.150.000,00                 #
#  Local:  Jardim La Salle - Toledo/PR             #
#  COD:    4301                                    #
####################################################


####################################################
#  Título: Apartamento - Padrão                    #
#  Valor:  Comprar R$ 450.000,00                   #
#  Local:  Jardim La Salle - Toledo/PR             #
#  COD:    3424                                    #
####################################################


#######################################

### No site https://www.imobiliariaativa.com.br/ 
como observamos no exemplo acima, ao buscarmos a classe desejada conseguimos obter os dados corretamente, pois muitas informações já estão presentes no HTML retornado pelo requests.

Contudo, alguns elementos específicos — como as imagens — são carregados dinamicamente via JavaScript, ou seja, não estão disponíveis diretamente no código HTML inicial da página, como podemos observar abaixo:

```js
function loadDynamicContent(elementId,pageName){
    var contentElement=$('#'+elementId);
    if(requisicaoFeita||!contentElement.length||contentElement.html()){
        return
    }
    $.ajax({url:pageName,type:'GET',dataType:"html"})